In [ ]:
# --- Cell 1: Setup - Mount Google Drive ---
# Description: This part mounts your Google Drive to the Colab environment,
# allowing access to your project files, data, and results directory.

print("--- [Cell 1] Mounting Google Drive ---")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Warning: Not in a Colab environment. Skipping Google Drive mount.")
except Exception as e:
    print(f"An error occurred while mounting Google Drive: {e}")

In [ ]:
# --- Cell 2: Setup - Install Dependencies ---
# Description: This installs all the necessary Python libraries required for
# the inference pipeline to run, such as transformers, torch, and scikit-image.

print("\n--- [Cell 2] Installing Dependencies ---")
import os
# Using os.system to run pip install. The -q flag makes the output less verbose.
os.system("pip install transformers torch torchvision Pillow scikit-image tqdm -q")
print("Dependencies installed.")


In [ ]:
# --- Cell 3: Configuration ---
# Description: Define all experiment parameters in this section. You can easily
# switch models, change paths, and adjust noise levels here before running.

print("\n--- [Cell 3] Setting up Configuration ---")
# Base path for your project in Google Drive
DRIVE_PROJECT_PATH = "your path here"

# --- Choose the model to run ---
# Options: 'qwen', 'smolvlm', 'phi3'
MODEL_NAME = 'qwen'

# --- Define paths ---
# Path to the JSONL file containing questions and image references
DATASET_PATH = "dataset path here"  # f"{DRIVE_PROJECT_PATH}data/iuxray_robustness.jsonl"

# Base path where the IU-Xray images are stored
IMAGE_BASE_PATH = "image path here" #f"{DRIVE_PROJECT_PATH}data/IU-Xray/images/"

prompt_style = "add_abstention_option" # or "default"

# Path to save the output logs. A new file will be created for each model.
RESULTS_DIR = f"{DRIVE_PROJECT_PATH}results/"
OUTPUT_LOG_PATH = f"{RESULTS_DIR}{MODEL_NAME}_iuxray_robustness_output_prompt_{prompt_style}.jsonl"




# --- Define noise levels ---
# 'original': runs on the clean images.
# 'gaussian_STDEV': Gaussian noise (e.g., 'gaussian_6' for std=6 on 0-255 scale)
# 'salt_pepper_AMOUNT': Salt & Pepper noise (e.g., 'salt_pepper_0.05')
# To replicate the CARES paper, use 'gaussian_6'
NOISE_TYPES = [
    'original',
    'gaussian_6'
    # You can add other noise types here for more experiments
    # 'salt_pepper_0.05',
]

# Pack configuration into a dictionary
experiment_config = {
    'model_name': MODEL_NAME,
    'dataset_path': DATASET_PATH,
    'image_base_path': IMAGE_BASE_PATH,
    'output_path': OUTPUT_LOG_PATH,
    'noise_types': NOISE_TYPES,
    'device': 'cuda', # or 'cpu'
    'prompt_style': prompt_style
}

# Ensure the results directory exists
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"Configuration set for model: {MODEL_NAME}")
print(f"Noise types to be tested: {NOISE_TYPES}")
print(f"Results will be saved to: {OUTPUT_LOG_PATH}")

In [ ]:
# --- Cell 4: Load Pipeline Code ---
# Description: This cell imports the main inference logic from the
# 'inference_pipeline.py' script. Make sure that script is in your DRIVE_PROJECT_PATH.

print("\n--- [Cell 4] Loading Inference Pipeline ---")
import sys
sys.path.append(DRIVE_PROJECT_PATH)

try:
    # Unload the module if it was already imported to pick up changes
    if 'inference_pipeline' in sys.modules:
        del sys.modules['inference_pipeline']

    from inference_pipeline import run_inference
    print("Successfully imported 'run_inference' from inference_pipeline.py")

except ImportError:
    print("Error: Could not find 'inference_pipeline.py'.")
    print(f"Please make sure the file is saved at: {DRIVE_PROJECT_PATH}inference_pipeline.py")
except Exception as e:
    print(f"An error occurred during import: {e}")
    # Set to None to prevent the next cell from running with an error
    run_inference = None

In [ ]:
# --- Cell 5: Run Inference ---
# Description: This final cell executes the inference pipeline using the
# configuration defined in Cell 3. It will log progress and save results
# continuously to the specified output file.

print("\n--- [Cell 5] Starting Inference ---")
if 'run_inference' in locals() and run_inference is not None:
    run_inference(experiment_config)
    print("\n--- Script Finished ---")
else:
    print("Skipping inference due to an error in the previous step.")

